# 02 - Data Cleaning & Harmonization

This notebook cleans and harmonizes raw USITC trade data for analysis.

**Key Steps:**
1. Load raw USITC data files (wide format)
2. Transform from wide to long format
3. Standardize country names
4. Apply inflation adjustment (convert to real dollars)
5. Calculate derived metrics (shares, growth rates)
6. Save processed dataset

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from data_loader import save_processed_data, DATA_RAW, DATA_PROCESSED, DATA_REFERENCE
from classification_mapper import standardize_country_names, add_historical_period
from transformers import (
    calculate_country_shares, 
    calculate_yoy_growth, 
    adjust_for_inflation,
    parse_monthly_columns,
    aggregate_monthly_to_annual,
    annualize_ytd_value
)
from usitc_api import USITCDataWebAPI

print("Modules loaded successfully")

## Step 1: Load Raw USITC Data Files

In [ ]:
# Define file paths
usitc_dir = DATA_RAW / 'usitc'

imports_file = usitc_dir / 'imports_1995_2024.csv'
exports_file = usitc_dir / 'exports_1995_2024.csv'

print(f"Imports file exists: {imports_file.exists()}")
print(f"Exports file exists: {exports_file.exists()}")

In [ ]:
# Load raw wide-format data
imports_wide = pd.read_csv(imports_file)
exports_wide = pd.read_csv(exports_file)

print(f"Imports: {imports_wide.shape[0]} countries, {imports_wide.shape[1]} columns")
print(f"Exports: {exports_wide.shape[0]} countries, {exports_wide.shape[1]} columns")

print("\nImports sample:")
display(imports_wide.head())

print("\nExports sample:")
display(exports_wide.head())

## Step 1b: Fetch Monthly Data from USITC API (2024-2025)

We fetch monthly data for recent years to capture 2025 YTD data that isn't in the annual files.

In [ ]:
# Initialize API client and fetch monthly data for 2024-2025
FETCH_FROM_API = True  # Set to False to skip API fetch and use cached files

if FETCH_FROM_API:
    try:
        api = USITCDataWebAPI()
        print("API client initialized successfully")
        
        # Fetch monthly imports
        print("\nFetching monthly imports for 2024-2025...")
        monthly_imports_raw = api.get_monthly_imports(start_year=2024, end_year=2025)
        print(f"Retrieved {len(monthly_imports_raw)} rows")
        
        # Save raw monthly data
        monthly_imports_path = usitc_dir / 'imports_monthly_2024_2025.csv'
        monthly_imports_raw.to_csv(monthly_imports_path, index=False)
        print(f"Saved to: {monthly_imports_path}")
        
        # Display sample
        display(monthly_imports_raw.head())
        
    except Exception as e:
        print(f"API fetch failed: {e}")
        print("Will attempt to load cached monthly data instead.")
        FETCH_FROM_API = False

# Load cached monthly data if API fetch was skipped or failed
if not FETCH_FROM_API:
    monthly_imports_path = usitc_dir / 'imports_monthly_2024_2025.csv'
    if monthly_imports_path.exists():
        monthly_imports_raw = pd.read_csv(monthly_imports_path)
        print(f"Loaded cached monthly data: {len(monthly_imports_raw)} rows")
        display(monthly_imports_raw.head())
    else:
        print("No cached monthly data found. Run with FETCH_FROM_API=True")

In [ ]:
# Parse monthly data to long format and aggregate to annual
# The parse_monthly_columns function handles 'Mon YYYY' format columns

monthly_imports_long = parse_monthly_columns(monthly_imports_raw)
print(f"Monthly imports (long format): {len(monthly_imports_long)} rows")
print(f"Months covered: {monthly_imports_long['month'].min()} to {monthly_imports_long['month'].max()}")
print(f"Years: {monthly_imports_long['year'].unique()}")

display(monthly_imports_long.head(10))

In [ ]:
# Aggregate monthly to annual totals
monthly_imports_annual = aggregate_monthly_to_annual(monthly_imports_long)
print(f"Aggregated to {len(monthly_imports_annual)} country-year records")

# Show 2025 YTD status
ytd_2025 = monthly_imports_annual[monthly_imports_annual['year'] == 2025]
print(f"\n2025 YTD data: {len(ytd_2025)} countries")
print(f"Months of 2025 data: {ytd_2025['month_count'].max() if len(ytd_2025) > 0 else 'N/A'}")
print(f"Last month: {ytd_2025['last_month'].max() if len(ytd_2025) > 0 else 'N/A'}")

display(monthly_imports_annual[monthly_imports_annual['year'] == 2025].head(10))

In [ ]:
# Prepare monthly data for merging with historical annual data
# Add trade_type column and format value
monthly_imports_annual['trade_type'] = 'import'

# Determine scaling: USITC API returns values in billions
# Convert to actual USD to match historical data
monthly_imports_annual['value'] = monthly_imports_annual['value'] * 1e9

# Keep only columns needed for merge
monthly_for_merge = monthly_imports_annual[['country', 'year', 'value', 'trade_type', 'is_ytd', 'month_count', 'last_month']].copy()

print(f"Monthly data ready for merge: {len(monthly_for_merge)} records")
display(monthly_for_merge[monthly_for_merge['year'] == 2025].nlargest(10, 'value'))

## Step 2: Transform from Wide to Long Format

In [ ]:
def wide_to_long(df_wide: pd.DataFrame, trade_type: str) -> pd.DataFrame:
    """
    Transform USITC wide-format data to long format.
    
    Args:
        df_wide: DataFrame with Country column and year columns (1995, 1996, ...)
        trade_type: 'import' or 'export'
    
    Returns:
        Long-format DataFrame with columns: country, year, value, trade_type
    """
    # Get the country column name (first column)
    country_col = df_wide.columns[0]
    
    # Get year columns (all numeric column names)
    year_cols = [c for c in df_wide.columns[1:] if str(c).replace('.', '').isdigit()]
    
    # Filter to only country and year columns
    df_subset = df_wide[[country_col] + year_cols].copy()
    
    # Melt from wide to long
    df_long = df_subset.melt(
        id_vars=[country_col],
        var_name='year',
        value_name='value'
    )
    
    # Rename country column
    df_long = df_long.rename(columns={country_col: 'country'})
    
    # Add trade type
    df_long['trade_type'] = trade_type
    
    # Convert year to integer
    df_long['year'] = df_long['year'].astype(int)
    
    # Convert value to numeric (handle any formatting issues)
    df_long['value'] = pd.to_numeric(
        df_long['value'].astype(str).str.replace(',', '').str.replace('"', ''),
        errors='coerce'
    )
    
    # Filter out Total row and other non-country rows
    exclude_rows = ['Total:', 'Total', 'Unspecified', 'Transshipment', 'Internat Organization']
    df_long = df_long[~df_long['country'].str.strip().isin(exclude_rows)]
    
    # Convert from billions to actual USD
    df_long['value'] = df_long['value'] * 1e9
    
    return df_long

# Transform both datasets
imports_long = wide_to_long(imports_wide, 'import')
exports_long = wide_to_long(exports_wide, 'export')

print(f"Imports (long): {len(imports_long):,} rows")
print(f"Exports (long): {len(exports_long):,} rows")

print("\nImports sample:")
display(imports_long.head(10))

In [ ]:
# Combine imports and exports into single DataFrame
trade_df = pd.concat([imports_long, exports_long], ignore_index=True)

# Add is_ytd flag (False for all historical annual data)
trade_df['is_ytd'] = False
trade_df['month_count'] = 12
trade_df['last_month'] = 12

print(f"Historical dataset: {len(trade_df):,} rows")
print(f"Year range: {trade_df['year'].min()} - {trade_df['year'].max()}")

# Filter historical data to years before the API data starts
# We'll use API data for 2024-2025 to get the most recent data
historical_cutoff = 2023
trade_df_historical = trade_df[trade_df['year'] <= historical_cutoff].copy()
print(f"\nHistorical data (up to {historical_cutoff}): {len(trade_df_historical):,} rows")

# Merge with monthly API data (2024-2025)
if 'monthly_for_merge' in dir() and len(monthly_for_merge) > 0:
    print(f"\nMerging with API data: {len(monthly_for_merge):,} rows")
    trade_df = pd.concat([trade_df_historical, monthly_for_merge], ignore_index=True)
    print(f"Combined dataset: {len(trade_df):,} rows")
else:
    print("\nNo API data to merge - using only historical data")
    trade_df = trade_df_historical

print(f"\nTrade type distribution:")
print(trade_df['trade_type'].value_counts())

print(f"\nFinal year range: {trade_df['year'].min()} - {trade_df['year'].max()}")
print(f"Unique countries: {trade_df['country'].nunique()}")

# Show 2025 data summary
if 2025 in trade_df['year'].values:
    ytd_info = trade_df[trade_df['year'] == 2025].iloc[0]
    print(f"\n2025 data: YTD through month {ytd_info['last_month']}")

## Step 3: Standardize Country Names

In [ ]:
# Apply country name standardization
trade_df = standardize_country_names(trade_df, country_col='country')

print(f"Unique countries after standardization: {trade_df['country'].nunique()}")

# Show top countries by total trade value
top_countries = trade_df.groupby('country')['value'].sum().sort_values(ascending=False).head(20)
print("\nTop 20 trading partners (total trade 1995-2024):")
for i, (country, value) in enumerate(top_countries.items(), 1):
    print(f"  {i:2}. {country}: ${value/1e12:.2f} trillion")

## Step 4: Apply Inflation Adjustment

In [ ]:
# Load GDP deflator
deflator_df = pd.read_csv(DATA_REFERENCE / 'gdp_deflator.csv')
print(f"GDP Deflator loaded: {len(deflator_df)} years")
display(deflator_df.tail(10))

In [ ]:
# Apply inflation adjustment (base year 2020)
trade_df = adjust_for_inflation(
    trade_df,
    deflator_df,
    value_col='value',
    year_col='year',
    base_year=2020
)

print("Sample with real values:")
sample = trade_df[trade_df['country'] == 'China'].sort_values('year')
display(sample[['year', 'country', 'trade_type', 'value', 'value_real']].head(10))

## Step 5: Calculate Derived Metrics

In [ ]:
# Calculate country shares (% of total trade by year and trade type)
trade_df = calculate_country_shares(
    trade_df,
    value_col='value_real',
    country_col='country',
    year_col='year',
    trade_type_col='trade_type'
)

# Show top import sources for the most recent year
latest_year = trade_df['year'].max()
imports_latest = trade_df[(trade_df['year'] == latest_year) & (trade_df['trade_type'] == 'import')]
imports_latest_top = imports_latest.nlargest(10, 'share_pct')[['country', 'value_real', 'share_pct']]

# Check if latest year is YTD
is_ytd = imports_latest['is_ytd'].iloc[0] if 'is_ytd' in imports_latest.columns else False
ytd_note = " (YTD)" if is_ytd else ""
print(f"Top import sources in {latest_year}{ytd_note} (by share):")
display(imports_latest_top)

In [ ]:
# Calculate year-over-year growth rates
trade_df = calculate_yoy_growth(
    trade_df,
    value_col='value_real',
    country_col='country',
    year_col='year',
    trade_type_col='trade_type'
)

print("China import growth over time:")
china_imports = trade_df[(trade_df['country'] == 'China') & (trade_df['trade_type'] == 'import')].sort_values('year')
display(china_imports[['year', 'value_real', 'share_pct', 'yoy_growth_pct']].tail(10))

In [ ]:
# Add historical period classification
trade_df = add_historical_period(trade_df, year_col='year')

print("Historical periods in data:")
print(trade_df.groupby('period')['year'].agg(['min', 'max', 'count']))

In [ ]:
# Final columns
print("Final columns:")
print(trade_df.columns.tolist())

print(f"\nFinal dataset shape: {trade_df.shape}")
print(f"Memory usage: {trade_df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

## Step 6: Data Validation

In [ ]:
# Check for missing values
print("Missing values:")
print(trade_df.isnull().sum())

# Check for negative values
print(f"\nNegative trade values: {(trade_df['value'] < 0).sum()}")

# Check year coverage
print(f"\nYears covered: {sorted(trade_df['year'].unique())}")

In [ ]:
# Verify total trade roughly matches known values
print("Total US imports by year (billions USD, nominal):")
annual_imports = trade_df[trade_df['trade_type'] == 'import'].groupby('year')['value'].sum() / 1e9
print(annual_imports.tail(10).round(1))

## Step 7: Save Processed Data

In [ ]:
# Ensure output directory exists
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Determine output filename based on data coverage
max_year = trade_df['year'].max()
output_filename = f'trade_data_1995_{max_year}.csv'
output_file = DATA_PROCESSED / output_filename

# Save combined processed data
trade_df.to_csv(output_file, index=False)
print(f"Saved processed data to: {output_file}")
print(f"File size: {output_file.stat().st_size / 1e6:.1f} MB")

# Check for YTD data
if trade_df['is_ytd'].any():
    ytd_years = trade_df[trade_df['is_ytd']]['year'].unique()
    for year in ytd_years:
        ytd_info = trade_df[(trade_df['year'] == year) & (trade_df['is_ytd'])].iloc[0]
        print(f"\nNote: {year} data is YTD through month {ytd_info['last_month']}")

# Also save separate imports and exports files
imports_processed = trade_df[trade_df['trade_type'] == 'import']
exports_processed = trade_df[trade_df['trade_type'] == 'export']

imports_processed.to_csv(DATA_PROCESSED / 'imports_processed.csv', index=False)
exports_processed.to_csv(DATA_PROCESSED / 'exports_processed.csv', index=False)

print(f"\nSaved {len(imports_processed):,} import records")
print(f"Saved {len(exports_processed):,} export records")

## Summary

In [ ]:
print("=" * 60)
print("DATA PROCESSING COMPLETE")
print("=" * 60)
print(f"\nTotal records: {len(trade_df):,}")
print(f"Years: {trade_df['year'].min()} - {trade_df['year'].max()}")
print(f"Countries: {trade_df['country'].nunique()}")
print(f"Trade types: {trade_df['trade_type'].unique().tolist()}")

# Report YTD status
if trade_df['is_ytd'].any():
    print(f"\nYTD data included:")
    for year in trade_df[trade_df['is_ytd']]['year'].unique():
        ytd_row = trade_df[(trade_df['year'] == year) & (trade_df['is_ytd'])].iloc[0]
        print(f"  - {year}: Through month {ytd_row['last_month']} ({ytd_row['month_count']} months)")

print(f"\nOutput files:")
print(f"  - {output_file}")
print(f"  - {DATA_PROCESSED / 'imports_processed.csv'}")
print(f"  - {DATA_PROCESSED / 'exports_processed.csv'}")
print("\n-> Proceed to 03_exploratory_analysis.ipynb")

# End of notebook